## Solving an optimization problem with `pyscipopt`

The package `pyscipopt` is a `python` wrapper for the [SCIP](https://www.scipopt.org/) optimization solver, which is capable of solving *Linear* and *Quadratic Programs* of both *continuous* and *integer variables*.

In [1]:
from pyscipopt import Model

### First example

Consider the basic optimization problem:
$$
\begin{array}{rl}
\min\quad & x + y \\
\text{s.t.}\quad 
& 2x + y \geq 5 \\
& x, y \geq 0 \\
& y \in \mathbb{Z}.
\end{array}
$$

In [2]:
# Create an empty model.
model = Model("Example")  # model name is optional

In [3]:
# Add variables.
x = model.addVar("x")
y = model.addVar("y", vtype="INTEGER")  # vtype -> variable type
# !!IMPORTANT!!  
# When lower bound is not set, it is automatically set to 0.0.
# If there is no lower bound, set it to `lb = None`.

# Add an objective function.
model.setObjective(x + y)

# Add constraints.
model.addCons(2*x + y >= 5)
model.addCons(x >= 0)
model.addCons(y >= 0)

c3

In [4]:
# Solve.
model.optimize()

# Get the solutions.
sol = model.getBestSol()

feasible solution found by trivial heuristic after 0.0 seconds, objective value 2.000000e+05
presolving:
(round 1, fast)       0 del vars, 2 del conss, 0 add conss, 2 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 2 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, fast)       1 del vars, 3 del conss, 0 add conss, 4 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (4 rounds: 4 fast, 1 medium, 1 exhaustive):
 2 deleted vars, 3 deleted constraints, 0 added constraints, 4 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
transformed 1/2 original solutions to the transformed problem space
Presolving Time: 0.00

SCIP Status        : problem is solved [optimal solution found]
Solving Time (sec) : 0.00
Solving Nodes      : 0
Primal Bound       : +2.50000000000000e+00 (2 solutions)
Dual Bound         : +2

In [5]:
print(sol)

{'y': -0.0, 'x': 2.5}


In [5]:
print(f"x: {sol[x]}, y: {sol[y]}")

x: 2.5, y: -0.0


----

### Another example

$$
\begin{array}{rl}
\min\quad & 2x + 3y - 5z \\
\text{s.t.}\quad & x + y \leq 5 \\
     & x + z \geq 3 \\
     & y + z = 4 \\
     & x,y,z \geq 0
\end{array}
$$


In [10]:
# Create another model.
mdl = Model()

# Add variables.
x = mdl.addVar(vtype='C', lb=0, ub=None, name='x')  # vtype='C' (for 'Continuous' = Real)
y = mdl.addVar(vtype='C', lb=0, ub=None, name='y')
z = mdl.addVar(vtype='C', lb=0, ub=None, name='z')

# Add constraints.
cons_1 = mdl.addCons(x + y <= 5, name="cons_1")
cons_2 = mdl.addCons(y + z >= 3, name="cons_2")
cons_3 = mdl.addCons(y + z == 4, name="cons_3")

# Add objective function.
mdl.setObjective(2 * x + 3 * y - 5 * z, sense="minimize")

In [12]:
# Solve.
mdl.optimize()

# Get solutions.
Sol = mdl.getBestSol()

# Get objective value.
Objval = mdl.getObjVal()

# Print solutions.
print(f"x: {Sol[x]}, y: {Sol[y]}, z: {Sol[z]}")
print(f"Optimal value: {Objval}") 

x: 2.0, y: 3.0, z: 0.0

x: 0.0, y: 0.0, z: 4.0
Optimal value: -20.0


---

### When there are many variables

$$
\begin{array}{rl}
\min \quad & x_{1} + x_{2} + \dots + x_{20} \\
\text{s.t.} \quad 
& x_{1} + x_{2} + \dots + x_{10} \geq 5 \\
& x_{5} + x_{6} + \dots + x_{15} = 4 \\
& x_{11} + x_{12} + \dots + x_{20} \leq 5 \\
& x_{2} + x_{4} + \dots + x_{20} = 6 \\
& 0 \leq x_{1},\dots,x_{20} \leq 3.
\end{array}
$$

In [13]:
from pyscipopt import quicksum

In [32]:
m = Model()

# add variables
x = [None for i in range(20)]
for i in range(20):
    x[i] = m.addVar(lb=0, ub=3, name=f"x_{i}")
    
# set objective function
m.setObjective( quicksum( x[i] for i in range(20) ) )

# add constraints
m.addCons( quicksum( x[i] for i in range(10) ) >= 5 )
m.addCons( quicksum( x[i] for i in range(4,15) ) == 4 )
m.addCons( quicksum( x[i] for i in range(10,20) ) <= 5 )
m.addCons( quicksum( x[2*i+1] for i in range(10) ) == 6)

c4

In [39]:
m.optimize()

presolving:
(round 1, fast)       2 del vars, 0 del conss, 0 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) symmetry computation finished: 11 generators found (max: 1500, log10 of symmetry group size: 0.0) (symcode time: 0.00)
dynamic symmetry handling statistics:
   orbitopal reduction:       no components
   orbital reduction:         no components
   lexicographic reduction:   no permutations
handled 7 out of 7 symmetry components
(round 2, exhaustive) 2 del vars, 0 del conss, 11 add conss, 0 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
presolving (3 rounds: 3 fast, 2 medium, 2 exhaustive):
 2 deleted vars, 0 deleted constraints, 11 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 18 variables (0 bin, 0 int, 0 impl, 18 c

In [40]:
sol = m.getBestSol()
print(sol)

{'x_0': 0.0, 'x_1': 2.0, 'x_2': 0.0, 'x_3': 0.0, 'x_4': 0.0, 'x_5': 1.5, 'x_6': 0.0, 'x_7': 1.5, 'x_8': 0.0, 'x_9': 0.0, 'x_10': 0.0, 'x_11': 1.0, 'x_12': 0.0, 'x_13': 0.0, 'x_14': 0.0, 'x_15': 0.0, 'x_16': 0.0, 'x_17': 0.0, 'x_18': 0.0, 'x_19': 0.0}


In [41]:
SOL = [sol[x[i]] for i in range(20)]
print(SOL)

[0.0, 2.0, 0.0, 0.0, 0.0, 1.5, 0.0, 1.5, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
